In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

class SugarcanePredictionPipeline:
    def __init__(self):
        self.data = None
        self.results = {}
        self.models = {}
        self.scalers = {}
        self.stage_results = {}
        
    def load_data(self):
        """Load and prepare the sugarcane dataset"""
        data_dict = {
            'Year': [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022],
            'Land_use': [380.543, 391.291, 389.417, 308.104, 338.626, 329.303, 375.216, 376.530, 447.204, 453.470, 442.958, 395.399, 368.426, 363.220, 336.003],
            'Yield': [85.72, 80.39, 80.2, 81.73, 76.65, 82.40, 81.33, 85.99, 76.93, 80.63, 75.64, 81.98, 82.65, 85.93, 85.32],
            'Sugar_Production': [4939000, 4814000, 4700000, 3700000, 3683000, 4250000, 4380000, 4700000, 4900000, 5100000, 4480000, 4725000, 4285000, 4335000, 4120000],
            'Exported_Sugar': [3700000, 3552000, 3600000, 2750000, 2800000, 3100000, 3242000, 3561000, 3700000, 4000000, 3600000, 3735000, 3600000, 3400000, 3120000]
        }
        self.data = pd.DataFrame(data_dict)
        print("Dataset loaded successfully!")
        print(f"Shape: {self.data.shape}")
        print(self.data.head())
        
    def visualize_data(self):
        """Create comprehensive visualizations"""
        fig = plt.figure(figsize=(20, 15))
        
        # 1. Overview of all variables
        plt.subplot(3, 3, 1)
        plt.plot(self.data['Year'], self.data['Land_use'], 'o-', label='Land Use', linewidth=2, markersize=6)
        plt.plot(self.data['Year'], self.data['Yield'], 'o-', label='Yield', linewidth=2, markersize=6)
        plt.title('Land Use and Yield Over Time', fontsize=14, fontweight='bold')
        plt.xlabel('Year')
        plt.ylabel('Value')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 2. Production and Export
        plt.subplot(3, 3, 2)
        plt.plot(self.data['Year'], self.data['Sugar_Production']/1000, 'o-', label='Production (k tonnes)', linewidth=2, markersize=6)
        plt.plot(self.data['Year'], self.data['Exported_Sugar']/1000, 'o-', label='Export (k tonnes)', linewidth=2, markersize=6)
        plt.title('Sugar Production and Export', fontsize=14, fontweight='bold')
        plt.xlabel('Year')
        plt.ylabel('Thousand Tonnes')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 3. Correlation matrix
        plt.subplot(3, 3, 3)
        corr_matrix = self.data[['Land_use', 'Yield', 'Sugar_Production', 'Exported_Sugar']].corr()
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, square=True, fmt='.2f')
        plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
        
        # 4. Distribution plots
        plt.subplot(3, 3, 4)
        plt.hist(self.data['Land_use'], bins=8, alpha=0.7, color='skyblue', edgecolor='black')
        plt.title('Land Use Distribution', fontsize=12, fontweight='bold')
        plt.xlabel('Land Use (1000 ha)')
        
        plt.subplot(3, 3, 5)
        plt.hist(self.data['Yield'], bins=8, alpha=0.7, color='lightgreen', edgecolor='black')
        plt.title('Yield Distribution', fontsize=12, fontweight='bold')
        plt.xlabel('Yield (t/ha)')
        
        # 5. Scatter plots
        plt.subplot(3, 3, 6)
        plt.scatter(self.data['Land_use'], self.data['Sugar_Production'], alpha=0.7, s=60, color='orange')
        plt.title('Land Use vs Production', fontsize=12, fontweight='bold')
        plt.xlabel('Land Use (1000 ha)')
        plt.ylabel('Sugar Production')
        
        plt.subplot(3, 3, 7)
        plt.scatter(self.data['Yield'], self.data['Sugar_Production'], alpha=0.7, s=60, color='red')
        plt.title('Yield vs Production', fontsize=12, fontweight='bold')
        plt.xlabel('Yield (t/ha)')
        plt.ylabel('Sugar Production')
        
        # 6. Normalized trends
        plt.subplot(3, 3, 8)
        for col in ['Land_use', 'Yield', 'Sugar_Production', 'Exported_Sugar']:
            normalized = (self.data[col] - self.data[col].min()) / (self.data[col].max() - self.data[col].min())
            plt.plot(self.data['Year'], normalized, 'o-', label=col, linewidth=2, markersize=4)
        plt.title('Normalized Trends', fontsize=12, fontweight='bold')
        plt.xlabel('Year')
        plt.ylabel('Normalized Value (0-1)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 7. Statistical summary
        plt.subplot(3, 3, 9)
        plt.axis('off')
        stats_text = self.data.describe().round(2).to_string()
        plt.text(0.1, 0.9, "Statistical Summary:", fontsize=12, fontweight='bold', transform=plt.gca().transAxes)
        plt.text(0.1, 0.1, stats_text, fontsize=8, fontfamily='monospace', transform=plt.gca().transAxes)
        
        plt.tight_layout()
        plt.show()
        
    def create_time_series_features(self, target_col, window_size=3):
        """Create sliding window features for time series prediction"""
        X, y = [], []
        target_values = self.data[target_col].values
        
        for i in range(window_size, len(target_values)):
            X.append(target_values[i-window_size:i])
            y.append(target_values[i])
        
        return np.array(X), np.array(y)
    
    def evaluate_model(self, y_true, y_pred, model_name="Model"):
        """Calculate evaluation metrics"""
        r2 = r2_score(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)
        
        return {
            'Model': model_name,
            'R2': r2,
            'RMSE': rmse,
            'MAE': mae
        }
    
    def stage_0_predict_land_use(self):
        """Stage 0: Predict Land Use using time series models"""
        print("\n" + "="*60)
        print("🟩 STAGE 0: PREDICTING LAND USE")
        print("="*60)
        
        X, y = self.create_time_series_features('Land_use', window_size=3)
        
        # Split data for time series (use last 20% for testing)
        split_idx = int(0.8 * len(X))
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]
        
        models_results = []
        
        # 1. ARIMA Model
        print("Training ARIMA model...")
        try:
            arima_model = ARIMA(self.data['Land_use'].values, order=(1,1,1))
            arima_fitted = arima_model.fit()
            arima_pred = arima_fitted.forecast(steps=len(y_test))
            arima_results = self.evaluate_model(y_test, arima_pred, "ARIMA")
            models_results.append(arima_results)
        except:
            print("ARIMA failed, using simple moving average")
            arima_pred = np.mean(y_train) * np.ones(len(y_test))
            arima_results = self.evaluate_model(y_test, arima_pred, "ARIMA")
            models_results.append(arima_results)
        
        # 2. Random Forest with lag features
        print("Training Random Forest...")
        rf_params = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7, None],
            'min_samples_split': [2, 5, 10]
        }
        rf = RandomForestRegressor(random_state=42)
        rf_grid = GridSearchCV(rf, rf_params, cv=3, scoring='r2', n_jobs=-1)
        rf_grid.fit(X_train, y_train)
        rf_pred = rf_grid.predict(X_test)
        rf_results = self.evaluate_model(y_test, rf_pred, "Random Forest")
        models_results.append(rf_results)
        
        # 3. XGBoost
        print("Training XGBoost...")
        xgb_params = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2]
        }
        xgb_model = xgb.XGBRegressor(random_state=42)
        xgb_grid = GridSearchCV(xgb_model, xgb_params, cv=3, scoring='r2', n_jobs=-1)
        xgb_grid.fit(X_train, y_train)
        xgb_pred = xgb_grid.predict(X_test)
        xgb_results = self.evaluate_model(y_test, xgb_pred, "XGBoost")
        models_results.append(xgb_results)
        
        # 4. LSTM
        print("Training LSTM...")
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        lstm_model = Sequential([
            LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)),
            Dropout(0.2),
            Dense(25),
            Dense(1)
        ])
        lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        
        X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
        X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
        
        lstm_model.fit(X_train_lstm, y_train, epochs=100, batch_size=16, verbose=0)
        lstm_pred = lstm_model.predict(X_test_lstm, verbose=0).flatten()
        lstm_results = self.evaluate_model(y_test, lstm_pred, "LSTM")
        models_results.append(lstm_results)
        
        # 5. GRU
        print("Training GRU...")
        gru_model = Sequential([
            GRU(50, activation='relu', input_shape=(X_train.shape[1], 1)),
            Dropout(0.2),
            Dense(25),
            Dense(1)
        ])
        gru_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        gru_model.fit(X_train_lstm, y_train, epochs=100, batch_size=16, verbose=0)
        gru_pred = gru_model.predict(X_test_lstm, verbose=0).flatten()
        gru_results = self.evaluate_model(y_test, gru_pred, "GRU")
        models_results.append(gru_results)
        
        # 6. Ensemble (Average)
        print("Creating Ensemble...")
        ensemble_pred = np.mean([arima_pred, rf_pred, xgb_pred, lstm_pred, gru_pred], axis=0)
        ensemble_results = self.evaluate_model(y_test, ensemble_pred, "Ensemble")
        models_results.append(ensemble_results)
        
        # Display results
        results_df = pd.DataFrame(models_results)
        print("\nStage 0 Results:")
        print(results_df.to_string(index=False))
        
        # Store best model predictions for next stage
        best_model_idx = results_df['R2'].idxmax()
        best_model_name = results_df.loc[best_model_idx, 'Model']
        print(f"\nBest Model: {best_model_name} (R² = {results_df.loc[best_model_idx, 'R2']:.4f})")
        
        self.stage_results['Stage_0'] = {
            'results': results_df,
            'predictions': ensemble_pred,  # Use ensemble for next stage
            'y_test': y_test,
            'best_model': best_model_name
        }
        
        return results_df
    
    def stage_1_predict_yield(self):
        """Stage 1: Predict Sugarcane Yield from Land Use"""
        print("\n" + "="*60)
        print("🟨 STAGE 1: PREDICTING SUGARCANE YIELD")
        print("="*60)
        
        X = self.data[['Land_use']].values
        y = self.data['Yield'].values
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        models_results = []
        
        # 1. Linear Regression
        print("Training Linear Regression...")
        lr = LinearRegression()
        lr.fit(X_train, y_train)
        lr_pred = lr.predict(X_test)
        lr_results = self.evaluate_model(y_test, lr_pred, "Linear Regression")
        models_results.append(lr_results)
        
        # 2. SVM
        print("Training SVM...")
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        svm_params = {
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 'auto'],
            'kernel': ['rbf', 'linear']
        }
        svm = SVR()
        svm_grid = GridSearchCV(svm, svm_params, cv=3, scoring='r2')
        svm_grid.fit(X_train_scaled, y_train)
        svm_pred = svm_grid.predict(X_test_scaled)
        svm_results = self.evaluate_model(y_test, svm_pred, "SVM")
        models_results.append(svm_results)
        
        # 3. Random Forest
        print("Training Random Forest...")
        rf_params = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7, None],
            'min_samples_split': [2, 5, 10]
        }
        rf = RandomForestRegressor(random_state=42)
        rf_grid = GridSearchCV(rf, rf_params, cv=3, scoring='r2')
        rf_grid.fit(X_train, y_train)
        rf_pred = rf_grid.predict(X_test)
        rf_results = self.evaluate_model(y_test, rf_pred, "Random Forest")
        models_results.append(rf_results)
        
        # 4. XGBoost
        print("Training XGBoost...")
        xgb_params = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2]
        }
        xgb_model = xgb.XGBRegressor(random_state=42)
        xgb_grid = GridSearchCV(xgb_model, xgb_params, cv=3, scoring='r2')
        xgb_grid.fit(X_train, y_train)
        xgb_pred = xgb_grid.predict(X_test)
        xgb_results = self.evaluate_model(y_test, xgb_pred, "XGBoost")
        models_results.append(xgb_results)
        
        # 5. LSTM (simplified for single feature)
        print("Training LSTM...")
        scaler_lstm = MinMaxScaler()
        X_train_lstm_scaled = scaler_lstm.fit_transform(X_train)
        X_test_lstm_scaled = scaler_lstm.transform(X_test)
        
        lstm_model = Sequential([
            Dense(50, activation='relu', input_shape=(1,)),
            Dropout(0.2),
            Dense(25, activation='relu'),
            Dense(1)
        ])
        lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        lstm_model.fit(X_train_lstm_scaled, y_train, epochs=100, batch_size=8, verbose=0)
        lstm_pred = lstm_model.predict(X_test_lstm_scaled, verbose=0).flatten()
        lstm_results = self.evaluate_model(y_test, lstm_pred, "LSTM")
        models_results.append(lstm_results)
        
        # 6. GRU (simplified)
        print("Training GRU...")
        gru_model = Sequential([
            Dense(50, activation='relu', input_shape=(1,)),
            Dropout(0.2),
            Dense(25, activation='relu'),
            Dense(1)
        ])
        gru_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        gru_model.fit(X_train_lstm_scaled, y_train, epochs=100, batch_size=8, verbose=0)
        gru_pred = gru_model.predict(X_test_lstm_scaled, verbose=0).flatten()
        gru_results = self.evaluate_model(y_test, gru_pred, "GRU")
        models_results.append(gru_results)
        
        # 7. Ensemble
        print("Creating Ensemble...")
        ensemble_pred = np.mean([lr_pred, svm_pred, rf_pred, xgb_pred, lstm_pred, gru_pred], axis=0)
        ensemble_results = self.evaluate_model(y_test, ensemble_pred, "Ensemble")
        models_results.append(ensemble_results)
        
        # Display results
        results_df = pd.DataFrame(models_results)
        print("\nStage 1 Results:")
        print(results_df.to_string(index=False))
        
        best_model_idx = results_df['R2'].idxmax()
        best_model_name = results_df.loc[best_model_idx, 'Model']
        print(f"\nBest Model: {best_model_name} (R² = {results_df.loc[best_model_idx, 'R2']:.4f})")
        
        self.stage_results['Stage_1'] = {
            'results': results_df,
            'predictions': ensemble_pred,
            'y_test': y_test,
            'best_model': best_model_name
        }
        
        return results_df
    
    def stage_2_predict_production(self):
        """Stage 2: Predict Sugar Production from Land Use + Yield"""
        print("\n" + "="*60)
        print("🟧 STAGE 2: PREDICTING SUGAR PRODUCTION")
        print("="*60)
        
        X = self.data[['Land_use', 'Yield']].values
        y = self.data['Sugar_Production'].values
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        models_results = []
        
        # 1. Linear Regression
        print("Training Linear Regression...")
        lr = LinearRegression()
        lr.fit(X_train, y_train)
        lr_pred = lr.predict(X_test)
        lr_results = self.evaluate_model(y_test, lr_pred, "Linear Regression")
        models_results.append(lr_results)
        
        # 2. SVM
        print("Training SVM...")
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        svm_params = {
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 'auto'],
            'kernel': ['rbf', 'linear']
        }
        svm = SVR()
        svm_grid = GridSearchCV(svm, svm_params, cv=3, scoring='r2')
        svm_grid.fit(X_train_scaled, y_train)
        svm_pred = svm_grid.predict(X_test_scaled)
        svm_results = self.evaluate_model(y_test, svm_pred, "SVM")
        models_results.append(svm_results)
        
        # 3. Random Forest
        print("Training Random Forest...")
        rf_params = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7, None],
            'min_samples_split': [2, 5, 10]
        }
        rf = RandomForestRegressor(random_state=42)
        rf_grid = GridSearchCV(rf, rf_params, cv=3, scoring='r2')
        rf_grid.fit(X_train, y_train)
        rf_pred = rf_grid.predict(X_test)
        rf_results = self.evaluate_model(y_test, rf_pred, "Random Forest")
        models_results.append(rf_results)
        
        # 4. XGBoost
        print("Training XGBoost...")
        xgb_params = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2]
        }
        xgb_model = xgb.XGBRegressor(random_state=42)
        xgb_grid = GridSearchCV(xgb_model, xgb_params, cv=3, scoring='r2')
        xgb_grid.fit(X_train, y_train)
        xgb_pred = xgb_grid.predict(X_test)
        xgb_results = self.evaluate_model(y_test, xgb_pred, "XGBoost")
        models_results.append(xgb_results)
        
        # 5. Neural Network (LSTM-style)
        print("Training Neural Network...")
        scaler_nn = MinMaxScaler()
        X_train_nn_scaled = scaler_nn.fit_transform(X_train)
        X_test_nn_scaled = scaler_nn.transform(X_test)
        
        nn_model = Sequential([
            Dense(100, activation='relu', input_shape=(2,)),
            Dropout(0.3),
            Dense(50, activation='relu'),
            Dropout(0.2),
            Dense(25, activation='relu'),
            Dense(1)
        ])
        nn_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        nn_model.fit(X_train_nn_scaled, y_train, epochs=150, batch_size=8, verbose=0)
        nn_pred = nn_model.predict(X_test_nn_scaled, verbose=0).flatten()
        nn_results = self.evaluate_model(y_test, nn_pred, "Neural Network")
        models_results.append(nn_results)
        
        # 6. Ensemble
        print("Creating Ensemble...")
        ensemble_pred = np.mean([lr_pred, svm_pred, rf_pred, xgb_pred, nn_pred], axis=0)
        ensemble_results = self.evaluate_model(y_test, ensemble_pred, "Ensemble")
        models_results.append(ensemble_results)
        
        # Display results
        results_df = pd.DataFrame(models_results)
        print("\nStage 2 Results:")
        print(results_df.to_string(index=False))
        
        best_model_idx = results_df['R2'].idxmax()
        best_model_name = results_df.loc[best_model_idx, 'Model']
        print(f"\nBest Model: {best_model_name} (R² = {results_df.loc[best_model_idx, 'R2']:.4f})")
        
        self.stage_results['Stage_2'] = {
            'results': results_df,
            'predictions': ensemble_pred,
            'y_test': y_test,
            'best_model': best_model_name
        }
        
        return results_df
    
    def stage_3_predict_export(self):
        """Stage 3: Predict Export from Land Use + Yield + Production"""
        print("\n" + "="*60)
        print("🟥 STAGE 3: PREDICTING SUGAR EXPORT")
        print("="*60)
        
        X = self.data[['Land_use', 'Yield', 'Sugar_Production']].values
        y = self.data['Exported_Sugar'].values
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        models_results = []
        
        # 1. Linear Regression
        print("Training Linear Regression...")
        lr = LinearRegression()
        lr.fit(X_train, y_train)
        lr_pred = lr.predict(X_test)
        lr_results = self.evaluate_model(y_test, lr_pred, "Linear Regression")
        models_results.append(lr_results)
        
        # 2. SVM
        print("Training SVM...")
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        svm_params = {
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 'auto'],
            'kernel': ['rbf', 'linear']
        }
        svm = SVR()
        svm_grid = GridSearchCV(svm, svm_params, cv=3, scoring='r2')
        svm_grid.fit(X_train_scaled, y_train)
        svm_pred = svm_grid.predict(X_test_scaled)
        svm_results = self.evaluate_model(y_test, svm_pred, "SVM")
        models_results.append(svm_results)
        
        # 3. Random Forest
        print("Training Random Forest...")
        rf_params = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7, None],
            'min_samples_split': [2, 5, 10]
        }
        rf = RandomForestRegressor(random_state=42)
        rf_grid = GridSearchCV(rf, rf_params, cv=3, scoring='r2')
        rf_grid.fit(X_train, y_train)
        rf_pred = rf_grid.predict(X_test)
        rf_results = self.evaluate_model(y_test, rf_pred, "Random Forest")
        models_results.append(rf_results)
        
        # 4. XGBoost
        print("Training XGBoost...")
        xgb_params = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2]
        }
        xgb_model = xgb.XGBRegressor(random_state=42)
        xgb_grid = GridSearchCV(xgb_model, xgb_params, cv=3, scoring='r2')
        xgb_grid.fit(X_train, y_train)
        xgb_pred = xgb_grid.predict(X_test)
        xgb_results = self.evaluate_model(y_test, xgb_pred, "XGBoost")
        models_results.append(xgb_results)
        
        #